## Import relevant libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import ignite
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

## Weighted MSE Loss

In [ ]:
def loss_weighted_mse(y_pred, y, loss_weights):
    y_diff_squared = (y - y_pred)**2
    y_diff_squared_weighted = torch.einsum('ijkl,j->ijkl', y_diff_squared, exp_mse_weights)
    weighted_mse = torch.mean(y_diff_squared_weighted)
    return weighted_mse

## Generate weights for exponential MSE Loss

In [ ]:
def generate_exp_weights(T, alpha):
    decay_length = 20.0
    augmentation_container = torch.linspace(0, 1, steps=T)
    augmentation_parameter = torch.linspace(0, alpha, steps=T)
    exp_mse_weights_unflipped = torch.exp(-decay_length * augmentation_container)
    exp_mse_weights = torch.flip(exp_mse_weights_unflipped, dims=[0])
    return exp_mse_weights

## PSF

### Gaussian PSF

In [ ]:
def generate_psf_kernel(sigma=10.0, psf_size=30):
#def generate_psf_kernel(sigma=1.0, psf_size=15):

    psf_kernel = torch.zeros(psf_size, psf_size)
    psf_center = psf_size // 2
    for x in range(psf_size):
        for y in range(psf_size):
            psf_kernel[x, y] = torch.exp(torch.tensor(-((x - psf_center) ** 2 + (y - psf_center) ** 2) / (2 * sigma ** 2)))
    psf_kernel /= psf_kernel.sum()

    return psf_kernel

### Literature PSF

In [ ]:
use_psf_lit = False
if use_psf_lit:
    psf_lit_image_path = "psf.png"
    psf_lit_image_pil = Image.open(psf_lit_image_path)
    psf_lit_kernel = transform(psf_lit_image_pil).squeeze()

### Choose PSF

In [ ]:
if use_psf_lit:
    psf_kernel = psf_lit_kernel
else:
    psf_kernel = generate_psf_kernel()

## Define CoordConv2D Layer

In [ ]:
class CoordConv2DLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input_tensor):
        _, _, height, width = input_tensor.size()

        # Create x and y coordinate grids
        xx_channel = torch.arange(width).view(1, 1, 1, width).expand(1, 1, height, width).float() / (width - 1)
        yy_channel = torch.arange(height).view(1, 1, height, 1).expand(1, 1, height, width).float() / (height - 1)

        # Concatenate the coordinate channels to the input tensor
        output_tensor = torch.cat([xx_channel, yy_channel], dim=1)
        return output_tensor

## Define FourierConv2D Layer

In [ ]:
class FourierConv2DLayer(nn.Module):
    def __init__(self, L):
        super().__init__()
        self.L = L

    def forward(self, x):
        _, num_input_channels, height, width = x.size()

        # Generate frequencies
        base_frequency = 2
        exponent_value = torch.arange(L)
        frequencies = torch.pow(torch.tensor(base_frequency), exponent_value).float()

        # Apply Fourier basis functions
        fourier_features = [torch.sin(frequencies[j] * torch.pi * x) for j in range(L)]
        fourier_features += [torch.cos(frequencies[j] * torch.pi * x) for j in range(L)]

        # Concatenate the Fourier features along the channel dimension
        fourier_features = torch.cat(fourier_features, dim=1)

        return fourier_features

## Define InverseConv2D Layer

In [ ]:
class InverseConv2DLayer(nn.Module):
    def __init__(self, L, num_standard_layers, max_reflectance, subsample_factor=2, use_fourier=True):
        super().__init__()
        self.L = L
        self.num_fourier_channels = 2
        self.num_standard_layers = num_standard_layers
        self.num_filters = 128
        self.use_fourier = use_fourier
        
        self.upsample_layer = nn.Upsample(scale_factor=subsample_factor, mode='nearest')
        self.coordinate_layer = CoordConv2DLayer()
        self.fourier_layer = FourierConv2DLayer(L)
            
        if self.use_fourier:
            self.fourier_layer = FourierConv2DLayer(L)
            self.num_fourier_channels = 4*L
        
        self.standard_hidden_layers = nn.ModuleList(
        [nn.Conv2d(self.num_fourier_channels, self.num_filters, kernel_size=3, padding='same')
         if c == 0 else
         nn.Conv2d(self.num_filters, self.num_filters, kernel_size=3, padding='same')
         for c in range(num_standard_layers)]
        )
        
        self.standard_output_layer = nn.Conv2d(self.num_filters, 1, kernel_size=3, padding='same')
        
        self.downsample_layer = nn.MaxPool2d(kernel_size=subsample_factor, stride=subsample_factor)
        
        # Initialize weights with He uniform variance scaling initializer
        for layer in self.standard_hidden_layers:
            nn.init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='leaky_relu')
            nn.init.zeros_(layer.bias)  # Initialize biases to zero
        nn.init.kaiming_uniform_(self.standard_output_layer.weight, mode='fan_in', nonlinearity='leaky_relu')
        nn.init.zeros_(self.standard_output_layer.bias)

    def forward(self, x):
        x = self.upsample_layer(x)
        output_coordinate = self.coordinate_layer(x)
        
        if self.use_fourier:
            output_fourier = self.fourier_layer(output_coordinate)
        else:
            output_fourier = output_coordinate
        
        x = output_fourier
        for layer in self.standard_hidden_layers:
            x = nn.functional.elu(layer(x))
        
        output = nn.functional.softplus(self.standard_output_layer(x))
        
        use_sigmoidal_output = True
        if use_sigmoidal_output:
            output = max_reflectance * torch.sigmoid(output)
        
        output_downsampled = self.downsample_layer(output)
        output_inverse = output_downsampled

        return output_coordinate, output_fourier, output, output_inverse

## Define Augmentation function

In [ ]:
def augment_image(original_image, augmentation_stride=1, blob_intensity=0.01, contrast_steps=2, mode="contrast"):
    
    augmented_image = torch.zeros_like(original_image)
    
    if mode == "translation":
        augmentation_stride = min(augmentation_stride, 19)
        augmented_image[:, :, :, augmentation_stride:] = original_image[:, :, :, :-augmentation_stride]
    elif mode == "crop":
        _, num_input_channels, height, width = original_image.size()
        win_size = 10
        #win_size = 50
        row_start = win_size
        row_end = height - win_size
        col_start = win_size
        col_end = width - win_size
        row_range = np.arange(row_start, row_end)
        col_range = np.arange(col_start, col_end)
        row, col = np.meshgrid(row_range, col_range, indexing='ij')
        idx = np.stack([row, col], axis=-1)
        idx = idx.reshape((-1, idx.shape[-1]))
        center = [idx_iter for iter, idx_iter in enumerate(idx)]
        #augmentation_idx = 10*(augmentation_stride - 1)
        augmentation_idx = augmentation_stride - 1
        #augmented_image = torch.zeros(1, num_input_channels, 2*win_size, 2*win_size)
        augmented_image[:, :, center[augmentation_idx][0]-win_size:center[augmentation_idx][0]+win_size, center[augmentation_idx][1]-win_size:center[augmentation_idx][1]+win_size] = original_image[:, :, center[augmentation_idx][0]-win_size:center[augmentation_idx][0]+win_size, center[augmentation_idx][1]-win_size:center[augmentation_idx][1]+win_size]
    elif mode == "elastic":
        elastic_parameter = 50.0 + augmentation_stride*10.0
        elastic_transformer = transforms.ElasticTransform(alpha=elastic_parameter)
        augmented_image = elastic_transformer(original_image)
    elif mode == "blob":
        blob_start = augmentation_stride
        blob_size = 5
        blob_end = blob_start + blob_size
        blob_tensor = torch.zeros(original_image.size())
        blob_tensor[:, :, blob_start:blob_end, blob_start:blob_end] = blob_intensity
        augmented_image = original_image + blob_tensor
    elif mode == "contrast":
        reflectance_max = float(torch.max(original_image).detach().numpy())
        reflectance_axis = torch.linspace(0, reflectance_max, steps=contrast_steps)
        reflectance_augmented = reflectance_axis[augmentation_stride-1]
        augmented_image = torch.abs(original_image - reflectance_augmented)
    elif mode == "contrast_and_blob":
        blob_start = augmentation_stride
        blob_size = 5
        blob_end = blob_start + blob_size
        blob_tensor = torch.zeros(original_image.size())
        blob_tensor[:, :, blob_start:blob_end, blob_start:blob_end] = blob_intensity
        reflectance_max = float(torch.max(original_image).detach().numpy())
        reflectance_axis = torch.linspace(0, reflectance_max, steps=contrast_steps)
        reflectance_augmented = reflectance_axis[augmentation_stride-1]
        augmented_image = torch.abs(original_image - reflectance_augmented) + blob_tensor
    elif mode == "background":
        background_max = float(torch.max(original_image).detach().numpy())
        background_axis = torch.linspace(0, background_max, steps=contrast_steps)
        background_augmented = background_axis[augmentation_stride-1]
        augmented_image = original_image + background_augmented
    
    return augmented_image

## Define AugmentationConv2D Layer

In [ ]:
class AugmentationConv2DLayer(nn.Module):
    def __init__(self, T):
        super().__init__()
        self.T = T

    def forward(self, x, alpha):
        _, num_input_channels, height, width = x.size()

        # Apply augmentation
        augmented_features = [
                augment_image(x, augmentation_stride=j+1, blob_intensity=alpha, contrast_steps=self.T) for j in range(self.T)
                ]

        # Concatenate the augmented features along the channel dimension
        augmented_features = torch.cat(augmented_features, dim=1)

        return augmented_features

## Define Microscope CNN Layer

In [ ]:
class MicroscopeCNNLayer(nn.Module):
    def __init__(self, psf_kernel):
        super().__init__()
        self.conv_layer = nn.Conv2d(1, 1, kernel_size=psf_kernel.size(0), padding='same', bias=False)
        self.conv_layer.weight = nn.Parameter(psf_kernel.unsqueeze(0).unsqueeze(0), requires_grad=False)

    def forward(self, x):
        output_conv = self.conv_layer(x)
        output_intensity = output_conv
        output_final = output_intensity / torch.max(output_intensity)
        return output_conv, output_intensity, output_final

## Define PINN

In [ ]:
class PINN(nn.Module):
    def __init__(self, L, T, alpha, num_standard_layers, max_reflectance, subsample_factor, psf_kernel, use_fourier=True):
        super().__init__()
        self.inverse_layer = InverseConv2DLayer(L, num_standard_layers, max_reflectance, subsample_factor, use_fourier=use_fourier)
        self.augmentation_layer = AugmentationConv2DLayer(T)
        self.forward_layer = MicroscopeCNNLayer(psf_kernel)

    def forward(self, x):
        output_coordinate, output_fourier, output, output_inverse = self.inverse_layer(x)
        output_augmentation = self.augmentation_layer(output_inverse, alpha)
        
        batch_size, num_augmentations, height, width = output_augmentation.size()
        
        output_conv = torch.zeros_like(output_augmentation)
        output_intensity = torch.zeros_like(output_augmentation)
        output_final = torch.zeros_like(output_augmentation)
        
        for t in range(self.augmentation_layer.T):
            output_conv_t, output_intensity_t, output_final_t = self.forward_layer(output_augmentation[:,t,:,:].unsqueeze(0))
            output_conv[:,t,:,:] = output_conv_t
            output_intensity[:,t,:,:] = output_intensity_t
            output_final[:,t,:,:] = output_final_t
        
        return output, output_final

## Sensor function

In [ ]:
def sensor_func(image, noise_level=0.1, subsample_factor=2):
    # Generate random noise with the same shape as the input image
    noise = noise_level * torch.randn_like(image)

    # Add the scaled noise to the original image
    noisy_image = image + noise
    
    # Apply subsampling using a pooling operation (e.g., MaxPool2d)
    subsampled_image = nn.functional.max_pool2d(noisy_image, kernel_size=subsample_factor, stride=subsample_factor)
    
    # Clip the values to ensure they are within the valid range (0, 1)
    sensor_image = torch.clamp(subsampled_image, 0, 1)

    return sensor_image

## Extract sample image

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

#sample_image_path = "shepp_logan_phantom.png"
#sample_image_path = "wavy_fibers_processed.png"
sample_image_path = "wavy_fibers_processed_2.png"
sample_image_pil = Image.open(sample_image_path)
sample_image_not_normalized = transform(sample_image_pil)
sample_image = sample_image_not_normalized / torch.max(sample_image_not_normalized)

## Visualize sample image

In [ ]:
plt.imshow(sample_image.squeeze(), cmap='gray')

## Construct Ground Truth Reflectance

In [ ]:
normalized_pixel_intensity = sample_image
alpha = 1.0
reflectance_ground_truth = alpha*normalized_pixel_intensity

## Resize Ground Truth Reflectance

In [ ]:
channels, height, width = reflectance_ground_truth.size()
reflectance_ground_truth = reflectance_ground_truth.view(1, channels, height, width)
reflectance_ground_truth.shape

## Generate "Training" Images

In [ ]:
apply_model_mismatch = False
if apply_model_mismatch:
    if use_psf_lit:
        psf_kernel_real = psf_lit_kernel
    else:
        psf_kernel_real = generate_psf_kernel(sigma=2.0, psf_size=21)
    microscope_model = MicroscopeCNNLayer(psf_kernel_real)
else:
    microscope_model = MicroscopeCNNLayer(psf_kernel)

subsample_factor = 2
apply_sensor = True

_, _, unaugmented_output = microscope_model(reflectance_ground_truth)
if apply_sensor:
    unaugmented_output = sensor_func(unaugmented_output, subsample_factor = subsample_factor)
else:
    subsample_factor = 1

#T = 2
#T = 5
T = 3

augmentation_layer = AugmentationConv2DLayer(T)
input_augmentation = augmentation_layer(reflectance_ground_truth, alpha)

batch_size, num_augmentations, height, width = input_augmentation.size()
training_image = torch.zeros(batch_size, num_augmentations, height//subsample_factor, width//subsample_factor)

use_output_augmentation = True
if use_output_augmentation:
    augmentation_axis = torch.linspace(0, alpha, steps=T)
    for t in range(T):
        augmentation_parameter = augmentation_axis[t]
        training_image_t = torch.abs(unaugmented_output - augmentation_parameter)
        training_image[:,t,:,:] = training_image_t
else: 
    for t in range(T):
        _, _, training_image_t = microscope_model(input_augmentation[:,t,:,:].unsqueeze(0))
        if apply_sensor:
            training_image_t = sensor_func(training_image_t, subsample_factor = subsample_factor)
        training_image[:,t,:,:] = training_image_t

## Check Training Image shape

In [ ]:
training_image.shape

## Visualize Training Images

In [ ]:
fig, axs = plt.subplots(1, T, figsize=(15, 3))
for t in range(T):
    axs[t].imshow(training_image[:,t,:,:].squeeze().detach().numpy(), cmap='gray')

In [ ]:
unaugmented_output.shape

In [ ]:
plt.imshow(training_image[:,0,:,:].squeeze().detach().numpy(), cmap='gray')

## Define SSIM Loss

In [ ]:
def ssim_loss(y_pred, y_true):
    
    ssim_metric = ignite.metrics.SSIM(data_range = 1.0)
    ssim_metric.update((y_pred, y_true))
    ssim_value = ssim_metric.compute()
    ssim_loss_value = 1.0 - ssim_value
    ssim_loss_tensor = torch.tensor(ssim_loss_value, requires_grad=True)
    
    return ssim_loss_tensor

In [ ]:
ssim_loss(training_image, training_image)

## Define PSNR

In [ ]:
def psnr(y_pred, y_true):
    
    psnr_metric = ignite.metrics.PSNR(data_range = 1.0)
    psnr_metric.update((y_pred, y_true))
    psnr_value = psnr_metric.compute()
    
    return psnr_value

## Define configurations for Inverse Layer

In [ ]:
L = 4
num_standard_layers = 4
max_reflectance = alpha
use_fourier = True

## Network training

### Model summary

In [ ]:
training_image.size()

In [ ]:
pinn_model_dummy = PINN(L, T, alpha, num_standard_layers, max_reflectance, subsample_factor, psf_kernel, use_fourier)
_, channels_dummy, height_dummy, width_dummy = training_image.size()
summary(pinn_model_dummy, input_size=(channels_dummy, height_dummy, width_dummy))

### Model training

In [ ]:
# Initialize PINN model
pinn_model = PINN(L, T, alpha, num_standard_layers, max_reflectance, subsample_factor, psf_kernel, use_fourier)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(pinn_model.parameters(), lr=1e-4)

# Set up the exponential learning rate scheduler
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=np.exp(np.log(5e-6 / 1e-4) / 10000))

# Set the regularization strengths (lambdas)
lambda_boundary = 0.25
lambda_structural = 0.01

# Number of training epochs
num_epochs = 10000

# Define list to store loss values
loss_list = []

# Define list to store intermediate outputs for reflectance
intermediate_output_reflectance = []

# Generate exp mse loss weights
use_exp_mse_weights = False
if use_exp_mse_weights:
    exp_mse_weights = generate_exp_weights(T, alpha)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    iter_reflectance, iter_image = pinn_model(training_image)

    # Calculate the mse loss
    if use_exp_mse_weights:
        loss_mse = loss_weighted_mse(iter_image, training_image, exp_mse_weights)
    else:
        loss_mse = criterion(iter_image, training_image)
        
    use_structural_regularization = False
    if use_structural_regularization:
        loss_ssim = ssim_loss(iter_image, training_image)
        loss_mse = loss_mse + lambda_structural * loss_ssim   
    
    # Calculate the boundary regularization loss
    loss_boundary = lambda_boundary * (
          torch.square(iter_image[:, :, 0, :] - 0).mean()
        + torch.square(iter_image[:, :, -1, :] - 0).mean()
        + torch.square(iter_image[:, :, :, 0] - 0).mean()
        + torch.square(iter_image[:, :, :, -1] - 0).mean()
    )
    
    # Calculate the total loss
    use_boundary_loss = False
    if use_boundary_loss:
        loss_total = loss_mse + loss_boundary
    else:
        loss_total = loss_mse
    loss = loss_total

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Update the learning rate
    scheduler.step()

    # Print training statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    loss_list.append(loss.item())

## Display Loss history

In [ ]:
plt.figure()
plt.plot(loss_list)
plt.xlabel('Iteration')
plt.ylabel('Training Loss')
plt.title('Training Loss History')

## Generate predictions

In [ ]:
predicted_reflectance, _ = pinn_model(training_image)
_, _, predicted_image = microscope_model(predicted_reflectance)

## Display results

In [ ]:
ground_truth_plot = reflectance_ground_truth.squeeze().numpy()
predicted_reflectance_plot = predicted_reflectance.squeeze().detach().numpy()
unaugmented_output_plot = unaugmented_output.squeeze().detach().numpy()

fig, (ax_1, ax_2, ax_3) = plt.subplots(1, 3, figsize=(14, 6))
fig.suptitle("Summary")

ax_1.imshow(ground_truth_plot, cmap='gray')
#ax_1.imshow(ground_truth_plot, cmap='inferno')
ax_1.set_ylabel('Y-coordinate')
ax_1.set_title('Ground Truth')

ax_2.imshow(unaugmented_output_plot, cmap='gray')
#ax_2.imshow(unaugmented_output_plot, cmap='inferno')
ax_2.set_title('Unaugmented Output')
ax_2.set_xlabel('X-coordinate')

ax_3.imshow(predicted_reflectance_plot, cmap='gray')
#ax_3.imshow(predicted_reflectance_plot, cmap='inferno')
ax_3.set_title('PINN Prediction')

plt.show()

In [ ]:
fig, (ax_1, ax_2) = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle("Visualize Reflectance")

#ax_1.imshow(ground_truth_plot, cmap='gray')
ax_1.imshow(ground_truth_plot, cmap='inferno')
ax_1.set_xlabel('X-coordinate')
ax_1.set_ylabel('Y-coordinate')
ax_1.set_title('Ground Truth')

#ax_2.imshow(predicted_reflectance_plot, cmap='gray')
ax_2.imshow(predicted_reflectance_plot, cmap='inferno')
ax_2.set_title('PINN Prediction')

In [ ]:
predicted_reflectance.shape, reflectance_ground_truth.shape

In [ ]:
ssim_loss(predicted_reflectance, reflectance_ground_truth[:, :, 1:, :])
#ssim_loss(predicted_reflectance, reflectance_ground_truth[:, :, :, 1:])
#ssim_loss(predicted_reflectance, reflectance_ground_truth)

In [ ]:
ground_truth_subsampled = nn.functional.max_pool2d(reflectance_ground_truth, kernel_size=subsample_factor, stride=subsample_factor)
ground_truth_subsampled.shape

In [ ]:
ssim_loss(unaugmented_output, ground_truth_subsampled)

In [ ]:
psnr(unaugmented_output, ground_truth_subsampled)

In [ ]:
psnr(predicted_reflectance, reflectance_ground_truth[:, :, :-1, :])
#psnr(predicted_reflectance, reflectance_ground_truth[:, :, :, 1:])
#psnr(predicted_reflectance, reflectance_ground_truth)

In [ ]:
predicted_image.shape

In [ ]:
predicted_reflectance.shape

In [ ]:
plt.imshow(predicted_image.squeeze().detach().numpy(), cmap='gray')

In [ ]:
x, y = torch.meshgrid(torch.arange(predicted_reflectance_plot.shape[0]), torch.arange(predicted_reflectance_plot.shape[1]))

fig= plt.figure(figsize=(10, 6))
fig.suptitle("Surface plot of the Predicted Reflectance")

ax_1 = fig.add_subplot(111, projection='3d')
surface = ax_1.plot_surface(x, y, predicted_reflectance_plot, cmap='viridis')
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')

plt.show()

In [ ]:
xg, yg = torch.meshgrid(torch.arange(ground_truth_plot.shape[0]), torch.arange(ground_truth_plot.shape[1]))

fig= plt.figure(figsize=(10, 6))
fig.suptitle("Surface plot of the Ground Truth")

ax_1 = fig.add_subplot(111, projection='3d')
surface = ax_1.plot_surface(xg, yg, ground_truth_plot, cmap='viridis')
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')

plt.show()

In [ ]:
predicted_reflectance_plot.shape

In [ ]:
ground_truth_plot.shape

In [ ]:
#error_plot = predicted_reflectance_plot - ground_truth_plot
error_plot = predicted_reflectance_plot - ground_truth_plot[1:, :]

In [ ]:
fig= plt.figure(figsize=(10, 6))
fig.suptitle("Surface plot of the Error")

ax_1 = fig.add_subplot(111, projection='3d')
surface = ax_1.plot_surface(x, y, error_plot, cmap='viridis')
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')

plt.show()

In [ ]:
plt.hist(error_plot)
plt.xlabel('error value')
plt.ylabel('frequency')
plt.title("Error Histogram")
plt.show()

In [ ]:
np.mean(np.square(error_plot))

In [ ]:
training_error_array = unaugmented_output - ground_truth_subsampled

In [ ]:
training_error_plot = training_error_array.squeeze().detach().numpy()

In [ ]:
plt.hist(training_error_plot)

In [ ]:
np.mean(np.square(training_error_plot))